#Title Here

Description: Here

## 0. Inport necessary library & Set workspace

In [1]:
import os
import numpy as np
import pandas as pd

from natsort import natsorted
from datetime import datetime
from dateutil.relativedelta import relativedelta

import math
import itertools
import pickle

# library for plotting
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow.compat.v1 as tf

from keras import backend as K
from keras.models import Model
from keras.preprocessing import sequence
from keras.utils.data_utils import Sequence
from keras.regularizers import l2
from keras.constraints import non_neg, Constraint
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve


from keras.layers import Input, Concatenate, Flatten
from keras.layers.core import Dense
from keras.layers.recurrent import LSTM
from tensorflow.keras.optimizers import Nadam, Adam, SGD, Adagrad
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization

from tensorflow.keras import utils as ku 
from nltk.util import ngrams

import keras
print(keras.__version__)


2.5.0


In [2]:
MY_WORKSPACE_DIR = os.path.join(os.getcwd(),"BPIC_11")
OUTPUT_DIR = os.path.join(MY_WORKSPACE_DIR, "output_files")

# The args dictionary is adapted from GenerativeLSTM by Manuel Camargo
# https://github.com/AdaptiveBProcess/GenerativeLSTM/blob/master/dg_training.py

args = dict()

args['processed_training_vec'] = os.path.join(OUTPUT_DIR, 'vec_training.p')
args['processed_test_vec'] = os.path.join(OUTPUT_DIR, 'vec_training.p')
args['weights'] = os.path.join(OUTPUT_DIR, 'weights.p')
args['indexes'] = os.path.join(OUTPUT_DIR, 'indexes.p')

args['log_name'] = 'bpic2011_Hospital_Data'
args['file_name'] = os.path.join(MY_WORKSPACE_DIR, 'BPIC_2011.csv') 
args['processed_file_name'] = os.path.join(OUTPUT_DIR, 'BPIC_2011_Processed.csv')
args['task']='outcome'

args['lstm_act'] = None # optimization function see keras doc
args['dense_act'] = None # optimization function see keras doc
args['n_size'] = 15 # n-gram size
args['l_size'] = 50 # LSTM layer sizes
args['norm_method'] = 'lognorm' # max, lognorm

In [3]:
args

{'processed_training_vec': 'c:\\Users\\mithr\\Documents\\GitHub\\Explainable-DL-ProcessPredictionss\\BPIC_11\\output_files\\vec_training.p',
 'processed_test_vec': 'c:\\Users\\mithr\\Documents\\GitHub\\Explainable-DL-ProcessPredictionss\\BPIC_11\\output_files\\vec_training.p',
 'weights': 'c:\\Users\\mithr\\Documents\\GitHub\\Explainable-DL-ProcessPredictionss\\BPIC_11\\output_files\\weights.p',
 'indexes': 'c:\\Users\\mithr\\Documents\\GitHub\\Explainable-DL-ProcessPredictionss\\BPIC_11\\output_files\\indexes.p',
 'log_name': 'bpic2011_Hospital_Data',
 'file_name': 'c:\\Users\\mithr\\Documents\\GitHub\\Explainable-DL-ProcessPredictionss\\BPIC_11\\BPIC_2011.csv',
 'processed_file_name': 'c:\\Users\\mithr\\Documents\\GitHub\\Explainable-DL-ProcessPredictionss\\BPIC_11\\output_files\\BPIC_2011_Processed.csv',
 'task': 'outcome',
 'lstm_act': None,
 'dense_act': None,
 'n_size': 15,
 'l_size': 50,
 'norm_method': 'lognorm'}

## 1. Data Cleaning

The [raw dataset](https://www.win.tue.nl/bpi/lib/exe/fetch.php?media=2011:hospital_log.csv.zip) is from Business Processing Intelligence Challenge (BPIC) 2011. Find more information [here](https://www.win.tue.nl/bpi/doku.php?id=2011:challenge).

In [4]:
# Dataframe creation
raw_data = pd.read_csv(args['file_name'], sep=";", low_memory=False)


df = raw_data
df.shape

(150291, 128)

In [5]:
pd.set_option('display.max_columns', None)
df.head()

,case:concept:name,event:concept:name,Treatment code,Producer code,Diagnosis,End date:10,End date:11,Diagnosis code,Number of executions,Start date:10,Start date:12,Start date:11,End date:12,End date:13,End date:14,End date:15,Specialism code:15,Specialism code:14,Specialism code:13,Specialism code:12,Diagnosis code:12,Start date:13,Diagnosis code:11,Start date:14,Start date:15,Diagnosis code:10,Specialism code:11,Specialism code:10,Treatment code:4,Treatment code:3,Diagnosis Treatment Combination ID:11,Treatment code:2,Treatment code:1,Diagnosis Treatment Combination ID:10,Diagnosis Treatment Combination ID:13,Diagnosis Treatment Combination ID:12,Diagnosis Treatment Combination ID:15,Diagnosis Treatment Combination ID:14,Treatment code:9,Treatment code:7,Treatment code:8,Treatment code:5,Treatment code:6,Treatment code:12,Treatment code:13,Treatment code:10,Treatment code:11,Treatment code:14,Treatment code:15,Specialism code:3,Start date:1,Specialism code:4,Specialism code:5,Activity code,Specialism code:6,Start date:5,Start date:4,Start date:3,Specialism code:1,Start date:2,Specialism code:2,Start date:9,Start date:8,Start date:7,Start date:6,Diagnosis:15,Diagnosis code:5,org:group,Specialism code:9,Diagnosis:14,Diagnosis code:4,Diagnosis code:3,Specialism code:7,Diagnosis code:2,Specialism code:8,Diagnosis:11,Diagnosis code:1,Diagnosis:10,Diagnosis:13,Diagnosis:12,Diagnosis code:9,Diagnosis code:8,Diagnosis code:7,Diagnosis code:6,lifecycle:transition,time:timestamp,End date,Age:5,Age:3,Age,Age:4,Diagnosis code:14,Specialism code,Diagnosis code:13,Diagnosis code:15,Diagnosis:8,Diagnosis:7,Diagnosis:6,Diagnosis:5,Diagnosis:4,Age:1,Diagnosis:3,Age:2,Diagnosis:2,Diagnosis:1,Diagnosis:9,Diagnosis Treatment Combination ID,Diagnosis Treatment Combination ID:7,Diagnosis Treatment Combination ID:8,Diagnosis Treatment Combination ID:9,Start date,Diagnosis Treatment Combination ID:6,Diagnosis Treatment Combination ID:5,Diagnosis Treatment Combination ID:4,Diagnosis Treatment Combination ID:3,Diagnosis Treatment Combination ID:2,Diagnosis Treatment Combination ID:1,End date:8,End date:7,End date:9,Section,End date:1,End date:2,End date:3,End date:4,End date:5,End date:6,Unnamed: 127
0,0,1e consult poliklinisch,103.0,SRTH,maligniteit cervix,NaN,NaN,M13,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005-01-03T00:14:24+01:00,NaN,NaN,410100,NaN,NaN,NaN,NaN,61.0,2005-01-03T00:14:24+01:00,61.0,NaN,NaN,NaN,NaN,NaN,NaN,Radiotherapy,NaN,NaN,NaN,NaN,NaN,106,NaN,NaN,106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,complete,2005-01-03T00:00:00+01:00,2006-01-04T23:45:36+01:00,NaN,NaN,33.0,NaN,NaN,61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gynaecologische tumoren,Gynaecologische tumoren,NaN,181229.0,NaN,NaN,NaN,2005-01-05T00:14:24+01:00,NaN,NaN,NaN,NaN,376908.0,376907.0,NaN,NaN,NaN,Section 5,2005-01-29T23:45:36+01:00,2005-01-29T23:45:36+01:00,NaN,NaN,NaN,NaN,NaN
1,0,administratief tarief - eerste pol,103.0,SRTH,maligniteit cervix,NaN,NaN,M13,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005-01-03T00:14:24+01:00,NaN,NaN,419100,NaN,NaN,NaN,NaN,61.0,2005-01-03T00:14:24+01:00,61.0,NaN,NaN,NaN,NaN,NaN,NaN,Radiotherapy,NaN,NaN,NaN,NaN,NaN,106,NaN,NaN,106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,complete,2005-01-03T00:00:00+01:00,2006-01-04T23:45:36+01:00,NaN,NaN,33.0,NaN,NaN,61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gynaecologische tumoren,Gynaecologische tumoren,NaN,181229.0,NaN,NaN,NaN,2005-01-05T00:14:24+01:00,NaN,NaN,NaN,NaN,376908.0,376907.0,NaN,NaN,NaN,Section 5,2005-01-29T23:45:36+01:00,2005-01-29T23:45:36+01:00,NaN,NaN,NaN,NaN,NaN
2,0,verlosk.-gynaec. korte kaart kosten-out,103.0,SGEH,maligniteit cervix,NaN,NaN,M13,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,13.0,NaN,NaN,NaN,NaN,NaN

### Delete irrelevant attributes

From the analysis of the columns for bpic11 and the standard of event log, we can divide attributes (columns) into `Trace` attributes and `Event` attributes. The `Trace` attributes share the same values in one case. While, the `Event` attributes keep changing between activities. Here list all the attribiutes for bpic11:

- Attributes for traces (Static)
    - case:concept:name
    - Age, Age:1-15
    - ~~Diagnosis, Diagnosis:1-15~~
    - Diagnosis code, Diagnosis code:1-15
    - Treatment code, Treatment code:1-15
    - ~~Diagnosis Treatment Combination ID, Diagnosis Treatment Combination ID:1-15~~
    - ~~Specialism code, Specialism code:1-15~~
    - ~~Start date, Start date:1-15~~
    - ~~End date, End date:1-15~~

- Attributes for events (Dynamic)
    - org:group
    - Number of executions
    - ~~Specialism code~~
    - event:concept:name
    - Producer code
    - Section
    - Activity code
    - time:timestamp
    - ~~lifecycle:transition~~

All irrelevant attributes with delete line need to be droped.

In [6]:
# Clean all columns with "Diagnosis", "Diagnosis Treatment Combination ID", "Specialism code" and "lifecycle:transition"
df = df.loc[:,~df.columns.str.startswith('Diagnosis:')]
df = df.drop(columns='Diagnosis')
df = df.loc[:,~df.columns.str.startswith('Specialism code')] 
df = df.loc[:,~df.columns.str.startswith('lifecycle:transition')]
df = df.loc[:,~df.columns.str.startswith('Diagnosis Treatment Combination ID')]

df = df.loc[:,~df.columns.str.startswith('Start date')] 
df = df.loc[:,~df.columns.str.startswith('End date')]

df = df.drop(columns='Unnamed: 127')
df.shape


(150291, 46)

### Combine repeating features to a list

In [7]:
# Sort all columns in alphabetical order

df = df.reindex(natsorted(df.columns), axis=1)
df.head()

,Activity code,Age,Age:1,Age:2,Age:3,Age:4,Age:5,Diagnosis code,Diagnosis code:1,Diagnosis code:2,Diagnosis code:3,Diagnosis code:4,Diagnosis code:5,Diagnosis code:6,Diagnosis code:7,Diagnosis code:8,Diagnosis code:9,Diagnosis code:10,Diagnosis code:11,Diagnosis code:12,Diagnosis code:13,Diagnosis code:14,Diagnosis code:15,Number of executions,Producer code,Section,Treatment code,Treatment code:1,Treatment code:2,Treatment code:3,Treatment code:4,Treatment code:5,Treatment code:6,Treatment code:7,Treatment code:8,Treatment code:9,Treatment code:10,Treatment code:11,Treatment code:12,Treatment code:13,Treatment code:14,Treatment code:15,case:concept:name,event:concept:name,org:group,time:timestamp
0,410100,33.0,NaN,NaN,NaN,NaN,NaN,M13,106,106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,SRTH,Section 5,103.0,13.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1e consult poliklinisch,Radiotherapy,2005-01-03T00:00:00+01:00
1,419100,33.0,NaN,NaN,NaN,NaN,NaN,M13,106,106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,SRTH,Section 5,103.0,13.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,administratief tarief - eerste pol,Radiotherapy,2005-01-03T00:00:00+01:00
2,10107,33.0,NaN,NaN,NaN,NaN,NaN,M13,106,106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,SGEH,Section 2,103.0,13.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,verlosk.-gynaec. korte kaart kosten-out,Nursing ward,2005-01-05T00:00:00+01:00
3,339486E,33.0,NaN,NaN,NaN,NaN,NaN,M13,106,106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,SGEC,Section 2,103.0,13.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,echografie - genitalia interna,Obstetrics & Gynaecology clinic,2005-01-05T00:00:00+01:00
4,410100,33.0,NaN,NaN,NaN,NaN,NaN,M13,106,106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,SGEH,Section 2,103.0,13.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1e consult poliklinisch,Nursing ward,2005-01-05T00:00:00+01:00


From the previous analysis, only three static `Trace` attributes - `Age`, `Diagnosis code` and `Treatment code` - contain repeating columns. Here we use functions for combining the repearting features into lists.

In [ ]:
# function for finding the valid value list

def findValidList(x):
    list_of_values = []
    for index, row in x.iterrows():
        values = []
        for i, v in row.items():
            if pd.notnull(v):
                values.append(v)
        values = frozenset(values)
        list_of_values.append(values)
    return list_of_values


In [ ]:
Age_List = findValidList(df.loc[:, df.columns.str.startswith('Age')])
DiagnosisCode_List = findValidList(df.loc[:, df.columns.str.startswith('Diagnosis code')])
TreatmentCode_List = findValidList(df.loc[:, df.columns.str.startswith('Treatment code')])

For three feature lists, we apply different methods to choose a suitable value.

- `Age`: Because the diagnosis and treatment for a patient may be over years, the age in one case may increase. Here we choose the latest one, which is also the largest one.

- `Diagnosis code`: This code shows the diagnosis from doctors. In the Appendix of bpic11 winner paper by Bose and Aalst, it provides a table for distribution of cases based on diagnosis code combinations. We choose top 8 combinations ({'M13'},{'M16'},{'M11'},{'M14'},{'106'},{'822', '106'}, {'M13', '106'}, {'M13', '822', '106'}) for our experimentation in order to avoid noise and unbalanced splits. 

- `Treatment code`: ...

```bib
@inproceedings{bpic11_winner,
  title={Analysis of patient treatment procedures: The BPI Challenge case study},
  author={R. P. J. C. Bose and W. Aalst},
  year={2011}
}
```

## 2. Understanding the dataset

### Data Analysis - Balance of Data Set

In [ ]:
#Checking the Balance of the Dataset, by the target variable
print('Distribution of cases by the target variable - diagnosis\n')
print(log_df.groupby(['diagnosis'])['caseid'].nunique())

We can see that there is an imbalance in the dataset, by the diagnosis.
Therefore, we can reduce the imbalance of data by grouping some of the diagnoses.

We use the logic below to perform the same, based on the paper by Bose and Aalst

```bib
@inproceedings{Bose2011AnalysisOP,
  title={Analysis of patient treatment procedures: The BPI Challenge case study},
  author={R. P. J. C. Bose and W. Aalst},
  year={2011}
}
```

* 821 group with m16
* 822 group with m13
* 823 group with 839 and discarded (minority class, multiple types of cancers)
* m12  discarded (minority class)
* m15 group with m14